### Lagrange interpolation approach

Consider gate $U(x) = e^{-ixG}$

$G$ has $n$ distinguish eigenvalues $\lambda_k, k=\{0,1,...,n-1\}$

$e^{-i x \boldsymbol{G}}=\sum_{k=0}^{n-1} e^{-i x \lambda_{k}} \prod_{l=0, l \neq k}^{n-1} \frac{\boldsymbol{G}-\lambda_{l} \boldsymbol{I}}{\lambda_{k}-\lambda_{l}} = f(\boldsymbol{G})=\Lambda_0 I + \Lambda_1 G + ... + \Lambda_{n-1} G^{n - 1}$

#### 1. Calculate coeff ${\Lambda_i}$ of a gate

Here we treat $\boldsymbol{G}$ as a variable. The input is only x and eigenvalues ${\lambda_i}$

In [1]:
import numpy as np
import base
# Input
x = -np.pi / 2
lambdas = [-5, -3, -1, 1, 3, 5]
# Polynomial presentation
fG = base.calculate_Lambda(lambdas, x)
# Output
print("Lambda_i: ", fG.coeff)


Lambda_i:  [(1.1481063742006435e-16+0j), (1.7333369499485123e-33+1.2416666666666667j), (-5.612964496092034e-17+8.673617379884035e-18j), (4.81482486096809e-34-0.24999999999999994j), (2.551347498223652e-18-6.505213034913027e-19j), (1.2037062152420224e-35+0.008333333333333335j)]


#### 2. Using more term to correct parameter-shift rule

In [2]:
alpha = np.pi / 4
# beta = np.pi / 2
# gamma = np.pi / 3
lambdas = [-1, 0, 1]
# lambdas = [-3/2, -1/2, 1/2, 3/2]
# lambdas = [-2, -1, 0, 1, 2]
# lambdas = [-3, -2, -1, 1, 2, 3]
# lambdas = [-3, -2, -1, 0, 1, 2, 5]
delta_Malpha = base.calculate_Lambda_matrix(
    lambdas, alpha) - base.calculate_Lambda_matrix(lambdas, -alpha)
# delta_Mbeta = base.calculate_Lambda_matrix(lambdas, beta) - base.calculate_Lambda_matrix(lambdas, -beta)
# delta_Mgamma = base.calculate_Lambda_matrix(lambdas, gamma) - base.calculate_Lambda_matrix(lambdas, -gamma)
d1 = 1/2
d2 = (-np.sqrt(2) + 1) / 4
# T_alpha = upper_matrix(delta_Malpha)
# T_beta = upper_matrix(delta_Mbeta)
# T_gamma = upper_matrix(delta_Mgamma)


In [3]:
print(np.round(delta_Malpha, 3))
print(base.upper_matrix(delta_Malpha))


[[ 0.+0.j     0.-1.414j  0.+0.j   ]
 [ 0.+1.414j  0.+0.j    -0.-0.414j]
 [ 0.+0.j    -0.+0.414j  0.+0.j   ]]
[[-0.-0.41421356j]]


In [2]:
Ts = []
deltas = []
thetas = []
dim_d = 8  # int(len(lambdas)**2/4) - 1
for i in range(0, dim_d):
    theta = np.random.uniform(0, 2*np.pi)
    delta = (base.calculate_Lambda_matrix(lambdas, theta) -
             base.calculate_Lambda_matrix(lambdas, -theta))
    thetas.append(theta)
    deltas.append(delta)
    Ts.append(base.upper_matrix(delta))


In [93]:
Ts = []
deltas = []
thetas = [np.pi/4, 3*np.pi/4]
dim_d = 2  # int(len(lambdas)**2/4) - 1
for i in range(0, dim_d):
    delta = (base.calculate_Lambda_matrix(
        lambdas, thetas[i]) - base.calculate_Lambda_matrix(lambdas, -thetas[i]))
    deltas.append(delta)
    Ts.append(base.upper_matrix(delta))


In [3]:
deltas[0]


array([[0.+0.00000000e+00j, 0.-2.19900459e-01j, 0.-2.40297202e-18j,
        0.+4.41680948e-02j, 0.+1.86027831e-19j, 0.-1.47006500e-03j],
       [0.+2.19900459e-01j, 0.+0.00000000e+00j, 0.-1.07316616e-01j,
        0.+2.58449455e-18j, 0.+4.87204939e-03j, 0.-3.23438466e-20j],
       [0.+2.40297202e-18j, 0.+1.07316616e-01j, 0.+0.00000000e+00j,
        0.-2.15550730e-02j, 0.-3.75464378e-20j, 0.+7.17426426e-04j],
       [0.-4.41680948e-02j, 0.-2.58449455e-18j, 0.+2.15550730e-02j,
        0.+0.00000000e+00j, 0.-9.78575214e-04j, 0.-1.07812822e-20j],
       [0.-1.86027831e-19j, 0.-4.87204939e-03j, 0.+3.75464378e-20j,
        0.+9.78575214e-04j, 0.+0.00000000e+00j, 0.-3.25703244e-05j],
       [0.+1.47006500e-03j, 0.+3.23438466e-20j, 0.-7.17426426e-04j,
        0.+1.07812822e-20j, 0.+3.25703244e-05j, 0.+0.00000000e+00j]])

In [4]:
T = Ts[0]
for i in range(1, len(Ts)):
    T = np.hstack((T, Ts[i]))


In [5]:
from scipy.linalg import null_space

init_rcond = 1
while True:
    d = (null_space(T, rcond=init_rcond))
    if d.shape[1] != 1:
        init_rcond /= 10
    else:
        break


In [6]:
d


array([[ 0.23239407-0.j        ],
       [ 0.33096666-0.31343884j],
       [ 0.5029375 -0.08534404j],
       [-0.25283961+0.04076971j],
       [ 0.24482923-0.06925397j],
       [-0.0987542 +0.16915642j],
       [ 0.36941991-0.05838874j],
       [ 0.35078611-0.21530309j]])

In [7]:
thetas


[4.6649932814646515,
 4.961862889137803,
 0.11643280295090416,
 4.066874818612189,
 1.9157607476229788,
 5.014346703733119,
 0.9976096238630942,
 1.373780860507946]

In [8]:
T @ d


array([[-6.54858112e-17-6.24500451e-17j],
       [ 1.97866896e-18+1.84314369e-18j],
       [ 9.71445147e-17+8.32667268e-17j],
       [-3.74049750e-18-4.77048956e-18j],
       [ 1.95156391e-17+1.90819582e-17j],
       [-5.28548559e-19-9.75781955e-19j],
       [ 7.04731412e-19+1.08420217e-18j],
       [ 3.04931861e-20+7.45388994e-20j]])

In [11]:
sumMatrix = d[0] * deltas[0]
for i in range(1, len(Ts)):
    sumMatrix += d[i] * deltas[i]
print(np.round(sumMatrix, 3))


[[ 0.+0.j     0.-0.009j  0.-0.j    -0.-0.j    -0.+0.j     0.+0.j   ]
 [-0.+0.009j  0.+0.j    -0.+0.j     0.-0.j     0.-0.j     0.-0.j   ]
 [-0.+0.j     0.-0.j     0.+0.j    -0.+0.j     0.-0.j     0.-0.j   ]
 [ 0.+0.j    -0.+0.j     0.-0.j     0.+0.j    -0.+0.j    -0.+0.j   ]
 [ 0.-0.j    -0.+0.j    -0.+0.j     0.-0.j     0.+0.j    -0.+0.j   ]
 [-0.-0.j    -0.+0.j    -0.+0.j     0.-0.j     0.-0.j     0.+0.j   ]]


Theo công thức phải chia thêm hệ số (2j)

In [113]:
d_t = d / ((2j)*(-1.09+0.38j))


In [114]:
d_t


array([[ 0.42691009+1.76683978e-04j],
       [-0.07324619-3.03141781e-05j]])

d_t giống với kết quả của [Pennylane](https://docs.pennylane.ai/en/stable/code/api/pennylane.CRY.html)

In [115]:
(1 - np.sqrt(2)) / (4*np.sqrt(2))


-0.07322330470336313

In [91]:
sumMatrix = d_t[0] * deltas[0]
for i in range(1, len(Ts)):
    sumMatrix += d_t[i] * deltas[i]
print(np.round(sumMatrix, 3))


[[ 0.+0.j    -1.+0.001j  0.+0.j   ]
 [ 1.-0.001j  0.+0.j     0.-0.j   ]
 [ 0.+0.j    -0.+0.j     0.+0.j   ]]


In [12]:
alpha = np.pi / 4
beta = 3*np.pi / 4
lambdas = [-1, 0, 1]

delta_Malpha = base.calculate_Lambda_matrix(
    lambdas, alpha) - base.calculate_Lambda_matrix(lambdas, -alpha)
delta_Mbeta = base.calculate_Lambda_matrix(
    lambdas, beta) - base.calculate_Lambda_matrix(lambdas, -beta)

d1 = (np.sqrt(2) + 1)/(4*np.sqrt(2))
d2 = (-np.sqrt(2) + 1)/(4*np.sqrt(2))

print(d1*delta_Malpha + d2*delta_Mbeta)


[[0.+0.00000000e+00j 0.-5.00000000e-01j 0.+0.00000000e+00j]
 [0.+5.00000000e-01j 0.+0.00000000e+00j 0.+5.55111512e-17j]
 [0.+0.00000000e+00j 0.-5.55111512e-17j 0.+0.00000000e+00j]]
